# <center> Process of cleaning and analyzing horse racing data from Kaggle dataset "Horses for Courses"</center>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

%matplotlib inline

#To supress scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
#There are a lot of issues with the 'position_two' column, so I left it out.
#Furthermore, the 'position_again' column is much more consistent and has all relevant win/place information  

fields = ["position_again","bf_odds","venue_name","date","market_name","condition","barrier","handicap_weight","last_five_starts","prize_money","sex","age","jockey","jockey_sex","trainer","days_since_last_run","overall_starts","overall_wins","overall_places","track_starts","track_wins","track_places","firm_starts","firm_wins","firm_places","good_starts","good_wins","good_places","slow_starts","slow_wins","slow_places","soft_starts","soft_wins","soft_places","heavy_starts","heavy_wins","heavy_places","distance_starts","distance_wins","distance_places"]

df = pd.read_csv("horses.csv", skipinitialspace=True, usecols=fields, low_memory=False)

df.head()

,position_again,bf_odds,venue_name,date,market_name,condition,barrier,handicap_weight,last_five_starts,prize_money,...,slow_places,soft_starts,soft_wins,soft_places,heavy_starts,heavy_wins,heavy_places,distance_starts,distance_wins,distance_places
0,1.00,2.88,Echuca,2016-06-27,R3 1200m Mdn,HVY9,3.00,58.50,f3,2160.00,...,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
1,2.00,15.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,10.00,58.50,x80x2,21175.00,...,1.00,2.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00
2,3.00,95.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,6.00,56.50,79x00,28855.00,...,0.00,5.00,0.00,0.00,3.00,0.00,0.00,4.00,0.00,0.00
3,nan,20.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,2.00,56.50,f0,1475.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,nan,2.74,Echuca,2016-06-27,R3 1200m Mdn,HVY9,7.00,56.50,f4x,1215.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [3]:
#Used to identify condition types
df.condition.value_counts(dropna=False)

GOOD4    81874
SOFT5    29009
GOOD3    28712
NaN      22188
SOFT6    15648
SOFT7    12246
HVY8     11269
AWT      10281
HVY10     4876
HVY9      4165
GOOD      1023
FIRM2      384
HEAVY        5
Name: condition, dtype: int64

In [4]:
df.date = pd.to_datetime(df.date, format='%Y'+'-'+'%m'+'-'+'%d')

#removes numbers from end of 'condition' string
df.condition = df.condition.str.replace('\d+', '')

#renaming condition values so that they're uniform
df.condition = df.condition.str.replace('HVY', 'HEAVY')
df.condition = df.condition.str.replace('AWT', 'GOOD') #AWT equates to a Good surface under some weather conditions

#reverses 'last_five_starts' (originally written right-to-left) so that it's easier to read in the future
df.last_five_starts = df.last_five_starts.str[::-1]

df.head()

,position_again,bf_odds,venue_name,date,market_name,condition,barrier,handicap_weight,last_five_starts,prize_money,...,slow_places,soft_starts,soft_wins,soft_places,heavy_starts,heavy_wins,heavy_places,distance_starts,distance_wins,distance_places
0,1.00,2.88,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,3.00,58.50,3f,2160.00,...,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
1,2.00,15.00,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,10.00,58.50,2x08x,21175.00,...,1.00,2.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00
2,3.00,95.00,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,6.00,56.50,00x97,28855.00,...,0.00,5.00,0.00,0.00,3.00,0.00,0.00,4.00,0.00,0.00
3,nan,20.00,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,2.00,56.50,0f,1475.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,nan,2.74,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,7.00,56.50,x4f,1215.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


## A prize_money_per_start column might be useful.

In [5]:
df.insert(10,'prize_money_per_start', df.prize_money/df.overall_starts)

df.drop('prize_money', axis=1, inplace=True)

df.head()

,position_again,bf_odds,venue_name,date,market_name,condition,barrier,handicap_weight,last_five_starts,prize_money_per_start,...,slow_places,soft_starts,soft_wins,soft_places,heavy_starts,heavy_wins,heavy_places,distance_starts,distance_wins,distance_places
0,1.00,2.88,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,3.00,58.50,3f,2160.00,...,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
1,2.00,15.00,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,10.00,58.50,2x08x,4235.00,...,1.00,2.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00
2,3.00,95.00,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,6.00,56.50,00x97,2885.50,...,0.00,5.00,0.00,0.00,3.00,0.00,0.00,4.00,0.00,0.00
3,nan,20.00,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,2.00,56.50,0f,1475.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,nan,2.74,Echuca,2016-06-27,R3 1200m Mdn,HEAVY,7.00,56.50,x4f,1215.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


## Creating general and conditions 'win_percent' and 'place_percent' columns:

In [6]:
#creates overall, track, and distance win_percent
#and place_percent columns and drops existing wins and places columns

columns_list = ["overall","track","distance"]

for x in columns_list:
    df[x+"_win_percent"] = df[x+"_wins"]/df[x+"_starts"]
    
    df[x+"_place_percent"] = df[x+"_places"]/df[x+"_starts"]
  
    df.drop([x+'_starts', x+'_wins', x+'_places'], axis=1, inplace=True)

In [7]:
#creates a win_percent and place_percent column for the current condition of the track

df.loc[df.condition.isna(), "condition_win_percent"] = np.nan

condition_list = ["firm","good","slow","soft","heavy"]

for x in condition_list:    
    
    df.loc[df.condition.str.lower() == x, "condition_win_percent"] = df[x+"_wins"]/df[x+"_starts"]
    
    df.loc[df.condition.str.lower() == x, "condition_place_percent"] = df[x+"_places"]/df[x+"_starts"]
    
    df.drop([x+'_starts', x+'_wins', x+'_places'], axis=1, inplace=True)

# Condition column is not necessary now that there are condition win and place percent columns 
df.drop('condition', axis=1, inplace=True)

# Replaces infinity (zero division) with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

## Cleaning data by removing races with missing win and/or place values in 'position_again' column:

In [8]:
#Used groupby to create indices by which to sort the re-indexed dataframe, 'df_indexed', below
df_grouped = df.groupby(['date','venue_name','market_name'])

#Drops all groups where there is no winners or places in 'position_again' column
index_list1 = df_grouped.position_again.sum(dropna=False).where(lambda x:(x == 3) | (x == 6)).dropna().index

df_indexed = df.set_index(['date','venue_name','market_name'])

df_cleaned1 = df_indexed.loc[index_list1]

In [9]:
df_grouped2 = df_cleaned1.groupby(['date','venue_name','market_name'])

#Regroups df_cleaned1 dataframe by same columns as before
#and shows that there are no duplicated values in the 'position_again' column for any race
index_list2 = df_grouped2.position_again.count().where(lambda x:x > 1).dropna().index

In [10]:
df_cleaned2 = df_cleaned1.loc[index_list2]

df_cleaned2

position_again  bf_odds  barrier  \
date       venue_name market_name                                        
2016-06-27 Echuca     R10 1400m Hcap            1.00    18.50    12.00   
                      R10 1400m Hcap            2.00    16.50     8.00   
                      R10 1400m Hcap            3.00    11.50     1.00   
                      R10 1400m Hcap             nan    19.00     5.00   
                      R10 1400m Hcap             nan     7.60     7.00   
                      R10 1400m Hcap             nan    10.00    11.00   
                      R10 1400m Hcap             nan     6.60     3.00   
                      R10 1400m Hcap             nan     4.20     2.00   
                      R10 1400m Hcap             nan    19.00    10.00   
                      R10 1400m Hcap             nan    65.00     9.00   
                      R10 1400m Hcap             nan    18.00    13.00   
                      R10 1400m Hcap             nan    70.00     6.00   
                      R10 1400m Hcap             nan    36.00     4.00   
                      R3 1200m Mdn              1.00     2.88     3.00   
                      R3 1200m Mdn              2.00    15.00    10.00   
                      R3 1200m Mdn              3.00    95.00     6.00   
                      R3 1200m Mdn               nan    20.00     2.00   
                      R3 1200m Mdn               nan     2.74     7.00   
                      R3 1200m Mdn               nan    17.50    11.00   
                      R3 1200m Mdn               nan    23.00     5.00   
                      R3 1200m Mdn               nan    18.50     1.00   
                      R3 1200m Mdn               nan    90.00     9.00   
                      R3 1200m Mdn               nan    46.00     4.00   
                      R3 1200m Mdn               nan   310.00     8.00   
                      R4 1400m 3yo              1.00     4.50     2.00   
                      R4 1400m 3yo              2.00     6.20     7.00   
                      R4 1400m 3yo              3.00     8.00     5.00   
                      R4 1400m 3yo               nan    10.00     8.00   
                      R4 1400m 3yo               nan    12.00    11.00   
                      R4 1400m 3yo               nan    36.00    10.00   
...                                              ...      ...      ...   
2018-07-14 Wyong      R1 1350m Mdn               nan    85.00     2.00   
                      R1 1350m Mdn               nan    12.50     5.00   
                      R1 1350m Mdn              1.00     2.50     4.00   
                      R1 1350m Mdn               nan     3.30     1.00   
                      R2 2100m Mdn              1.00    14.50     5.00   
                      R2 2100m Mdn               nan    55.00     1.00   
                      R2 2100m Mdn              2.00     2.96     2.00   
                      R2 2100m Mdn               nan     6.20     6.00   
                      R2 2100m Mdn               nan    46.00     4.00   
                      R2 2100m Mdn               nan    20.00     7.00   
                      R2 2100m Mdn               nan     2.74     3.00   
                      R3 1200m 2yo               nan   180.00     6.00   
                      R3 1200m 2yo               nan     6.80     5.00   
                      R3 1200m 2yo               nan    11.50     2.00   
                      R3 1200m 2yo               nan     5.80     9.00   
                      R3 1200m 2yo               nan     5.20     7.00   
                      R3 1200m 2yo              2.00    22.00     3.00   
                      R3 1200m 2yo               nan   370.00    11.00   
                      R3 1200m 2yo               nan    32.00     8.00   
                      R3 1200m 2yo              1.00     6.60    10.00   
                      R3 1200m 2yo               nan   130.00    13.00   
                      R3 1200m 2yo        

In [27]:
df_grouped3 = df_cleaned2.groupby(['date','venue_name','market_name'])

# Number of usable races:
len(df_grouped3)

22125

## Next, I create an 'average_weight' column for each race to create a weight_minus_average column and drop the handicap_weight column.

In [12]:
#Creates weight_minus_average column and drops 'handicap_weight','average_weight' columns
average_weight = df_grouped3.handicap_weight.transform('mean')

df_cleaned2.insert(4,'average_weight',average_weight)

df_cleaned2.insert(5,'weight_minus_average', df_cleaned2.handicap_weight - df_cleaned2.average_weight)

df_cleaned2.drop(['handicap_weight','average_weight'], axis=1, inplace=True)

df_cleaned2

position_again  bf_odds  barrier  \
date       venue_name market_name                                        
2016-06-27 Echuca     R10 1400m Hcap            1.00    18.50    12.00   
                      R10 1400m Hcap            2.00    16.50     8.00   
                      R10 1400m Hcap            3.00    11.50     1.00   
                      R10 1400m Hcap             nan    19.00     5.00   
                      R10 1400m Hcap             nan     7.60     7.00   
                      R10 1400m Hcap             nan    10.00    11.00   
                      R10 1400m Hcap             nan     6.60     3.00   
                      R10 1400m Hcap             nan     4.20     2.00   
                      R10 1400m Hcap             nan    19.00    10.00   
                      R10 1400m Hcap             nan    65.00     9.00   
                      R10 1400m Hcap             nan    18.00    13.00   
                      R10 1400m Hcap             nan    70.00     6.00   
                      R10 1400m Hcap             nan    36.00     4.00   
                      R3 1200m Mdn              1.00     2.88     3.00   
                      R3 1200m Mdn              2.00    15.00    10.00   
                      R3 1200m Mdn              3.00    95.00     6.00   
                      R3 1200m Mdn               nan    20.00     2.00   
                      R3 1200m Mdn               nan     2.74     7.00   
                      R3 1200m Mdn               nan    17.50    11.00   
                      R3 1200m Mdn               nan    23.00     5.00   
                      R3 1200m Mdn               nan    18.50     1.00   
                      R3 1200m Mdn               nan    90.00     9.00   
                      R3 1200m Mdn               nan    46.00     4.00   
                      R3 1200m Mdn               nan   310.00     8.00   
                      R4 1400m 3yo              1.00     4.50     2.00   
                      R4 1400m 3yo              2.00     6.20     7.00   
                      R4 1400m 3yo              3.00     8.00     5.00   
                      R4 1400m 3yo               nan    10.00     8.00   
                      R4 1400m 3yo               nan    12.00    11.00   
                      R4 1400m 3yo               nan    36.00    10.00   
...                                              ...      ...      ...   
2018-07-14 Wyong      R1 1350m Mdn               nan    85.00     2.00   
                      R1 1350m Mdn               nan    12.50     5.00   
                      R1 1350m Mdn              1.00     2.50     4.00   
                      R1 1350m Mdn               nan     3.30     1.00   
                      R2 2100m Mdn              1.00    14.50     5.00   
                      R2 2100m Mdn               nan    55.00     1.00   
                      R2 2100m Mdn              2.00     2.96     2.00   
                      R2 2100m Mdn               nan     6.20     6.00   
                      R2 2100m Mdn               nan    46.00     4.00   
                      R2 2100m Mdn               nan    20.00     7.00   
                      R2 2100m Mdn               nan     2.74     3.00   
                      R3 1200m 2yo               nan   180.00     6.00   
                      R3 1200m 2yo               nan     6.80     5.00   
                      R3 1200m 2yo               nan    11.50     2.00   
                      R3 1200m 2yo               nan     5.80     9.00   
                      R3 1200m 2yo               nan     5.20     7.00   
                      R3 1200m 2yo              2.00    22.00     3.00   
                      R3 1200m 2yo               nan   370.00    11.00   
                      R3 1200m 2yo               nan    32.00     8.00   
                      R3 1200m 2yo              1.00     6.60    10.00   
                      R3 1200m 2yo               nan   130.00    13.00   
                      R3 1200m 2yo        

In [13]:
df_grouped3 = df_cleaned2.groupby(['date','venue_name','market_name'])

## The first thing to do is test which features are significant.

### For jockey gender:

#### Overall percentage of men and women in races where both are represented:

In [15]:
#Drops races where there is only one jockey gender
jockey_sex_population = df_grouped3.jockey_sex.value_counts(normalize=True,dropna=False).where(lambda x: x != 1).dropna()

jockey_sex_population.mean(level=3)

jockey_sex
Male     0.75
Female   0.25
Name: jockey_sex, dtype: float64

In [16]:
jockey_sex_population_indices = jockey_sex_population.index

jockey_sex_population_cleaned = df_cleaned1.loc[jockey_sex_population_indices].drop_duplicates()

#### Win percentage of those races:

In [19]:
#Isolates wins in races with both jockey genders represented
jockey_sex_win_indices = jockey_sex_population_cleaned.position_again.apply(lambda x: x == 1)

jockey_sex_population_cleaned[jockey_sex_win_indices].jockey_sex.value_counts(normalize=True)

Male     0.79
Female   0.21
Name: jockey_sex, dtype: float64

#### Ensuring that races counted were same for both percentage values:

In [26]:
#Number of races for both previous percentages
#followed by the number of races there should be (given by number of first-place finishes): 

print(len(jockey_sex_population_cleaned.groupby(['date','venue_name','market_name'])))

print(len(jockey_sex_population_cleaned[jockey_sex_win_indices].jockey_sex.groupby(['date','venue_name','market_name'])))

jockey_sex_population_cleaned.position_again.value_counts()

16630
16630


2.00    16630
1.00    16630
3.00    14278
Name: position_again, dtype: int64

### Using a 2 sample proportion z-test, I find that 

In [ ]:
n = 
pnull = 
phat =